In [72]:
import numpy as np
import pandas as pd

from glob import glob

In [73]:
year = 2013
election_round = "first_round" # first_round, runoff

In [74]:
if year == 2013:
    output = []
    for file in glob(f"data/Chile/{year}_{election_round}_*.xlsx"):
        region_id = file.replace(f"data/Chile\{year}_{election_round}_", "")[0:2]
        tmp = pd.read_excel(file)
        tmp["region_id"] = region_id
        output.append(tmp)

    df = pd.concat(output, ignore_index=True)
else:
    df = pd.read_excel(f"data/Chile/{year}_{election_round}.xlsx")

df.head()

,Provincia,Circ. Senatorial,Distrito,Comuna,Circ. Electoral,Nro. Mesa,Tipo Mesa,Nro. En Voto,Partido,Candidato,Votos TRICEL,region_id
0,IQUIQUE,1.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1.0,INDEPENDIENTES,FRANCO ALDO PARISI FERNANDEZ,20.0,01
1,IQUIQUE,1.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,2.0,PARTIDO HUMANISTA,MARCEL CLAUDE REYES,2.0,01
2,IQUIQUE,1.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,3.0,PARTIDO REGIONALISTA DE LOS INDEPENDIENTES,RICARDO ISRAEL ZIPPER,2.0,01
3,IQUIQUE,1.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,4.0,PARTIDO PROGRESISTA,MARCO ENRIQUEZ-OMINAMI GUMUCIO,15.0,01
4,IQUIQUE,1.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,5.0,IGUALDAD,ROXANA MIRANDA MENESES,3.0,01


In [75]:
df.loc[df["Provincia"] == "ÑUBLE", "region_id"] = 16
df.loc[df["Provincia"] == "ÑUBLE", "Región"] = 16

In [76]:
# list(df)

In [77]:
if year == 2013:
    df["polling_id"] = df["Comuna"] + "-" + df["Circ. Electoral"] + "-" + df["Nro. Mesa"].fillna(0).astype(int).astype(str)\
        + "-" + df["Tipo Mesa"].fillna("NA")

elif year == 2017:
    df["polling_id"] = df["Comuna"] + "-" + df["Circ.Electoral"] + "-" + df["Local"] + "-" + df["Mesa"].fillna(0).astype(int).astype(str)\
        + "-" + df["Tipo mesa"].fillna("NA") + "-" + df["Mesas Fusionadas"].astype(str)

elif year == 2021:
    df["polling_id"] = df["Comuna"] + "-" + df["Circ. Electoral"] + "-" + df["Local"] + "-" + df["Nro. Mesa"].fillna(0).astype(int).astype(str)\
        + "-" + df["Tipo de mesa"].fillna("NA").astype(str) + "-" + df["Mesas Fusionadas"].astype(str)

In [78]:
df.Candidato.unique()

array(['FRANCO ALDO PARISI FERNANDEZ', 'MARCEL CLAUDE REYES',
       'RICARDO ISRAEL ZIPPER', 'MARCO ENRIQUEZ-OMINAMI GUMUCIO',
       'ROXANA MIRANDA MENESES', 'MICHELLE BACHELET JERIA',
       'EVELYN MATTHEI FORNET', 'ALFREDO SFEIR YOUNIS',
       'TOMAS JOCELYN-HOLT LETELIER', 'VOTOS NULOS', 'VOTOS EN BLANCO',
       nan, 'TOTALES'], dtype=object)

In [79]:
df = df[~df["Candidato"].isin(["VOTOS NULOS", "VOTOS EN BLANCO", "Votos Blancos", "Votos Nulos", "TOTALES", np.nan])]

In [80]:
df = df.dropna(axis=1, how="all")

df["Votos TRICEL"] = df["Votos TRICEL"].fillna(0)
df["rank"] = df.groupby(["polling_id"])["Votos TRICEL"].rank("min", ascending=False).astype(int)

In [81]:
tt = df.groupby(["polling_id", "Candidato"]).agg({"Votos TRICEL": "sum"})
tt["rate"] = tt.groupby(level=[0]).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt = tt[["polling_id", "Candidato", "rate"]]

C:\Users\cnava\AppData\Local\Temp\ipykernel_45680\411107196.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  tt["rate"] = tt.groupby(level=[0]).apply(lambda x: x/x.sum())


In [82]:
df = pd.merge(df, tt, on=["polling_id", "Candidato"])

In [83]:
df_filtered = df[["polling_id", "Candidato", "Votos TRICEL", "rank", "rate"]]\
    .rename(columns={"Candidato": "candidate", "Votos TRICEL": "value"})

In [84]:
df_filtered["candidate"] = df_filtered["candidate"].apply(lambda x: x.strip())

In [85]:
df_filtered.candidate.unique()

array(['FRANCO ALDO PARISI FERNANDEZ', 'MARCEL CLAUDE REYES',
       'RICARDO ISRAEL ZIPPER', 'MARCO ENRIQUEZ-OMINAMI GUMUCIO',
       'ROXANA MIRANDA MENESES', 'MICHELLE BACHELET JERIA',
       'EVELYN MATTHEI FORNET', 'ALFREDO SFEIR YOUNIS',
       'TOMAS JOCELYN-HOLT LETELIER'], dtype=object)

In [86]:
print(df_filtered.shape)
df_filtered.head()

(372141, 5)


,polling_id,candidate,value,rank,rate
0,ALTO HOSPICIO-ALTO HOSPICIO-1-M,FRANCO ALDO PARISI FERNANDEZ,20.0,3,0.120482
1,ALTO HOSPICIO-ALTO HOSPICIO-1-M,MARCEL CLAUDE REYES,2.0,6,0.012048
2,ALTO HOSPICIO-ALTO HOSPICIO-1-M,RICARDO ISRAEL ZIPPER,2.0,6,0.012048
3,ALTO HOSPICIO-ALTO HOSPICIO-1-M,MARCO ENRIQUEZ-OMINAMI GUMUCIO,15.0,4,0.090361
4,ALTO HOSPICIO-ALTO HOSPICIO-1-M,ROXANA MIRANDA MENESES,3.0,5,0.018072


In [87]:
df_filtered.to_csv(f"data_output/Chile/{year}_{election_round}.csv.gz", compression="gzip", index=False)

In [88]:
list(df)

['Provincia',
 'Circ. Senatorial',
 'Distrito',
 'Comuna',
 'Circ. Electoral',
 'Nro. Mesa',
 'Tipo Mesa',
 'Nro. En Voto',
 'Partido',
 'Candidato',
 'Votos TRICEL',
 'region_id',
 'Región',
 'polling_id',
 'rank',
 'rate']

In [89]:
# df[["polling_id", "Región", "Provincia", "Circ.Senatorial", "Distrito", "Comuna", "Circ.Electoral", "Local",
#         "Mesa", "Tipo mesa", "Mesas Fusionadas", "Electores"]].drop_duplicates()

In [90]:
df

,Provincia,Circ. Senatorial,Distrito,Comuna,Circ. Electoral,Nro. Mesa,Tipo Mesa,Nro. En Voto,Partido,Candidato,Votos TRICEL,region_id,Región,polling_id,rank,rate
0,IQUIQUE,1.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1.0,INDEPENDIENTES,FRANCO ALDO PARISI FERNANDEZ,20.0,01,NaN,ALTO HOSPICIO-ALTO HOSPICIO-1-M,3,0.120482
1,IQUIQUE,1.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,2.0,PARTIDO HUMANISTA,MARCEL CLAUDE REYES,2.0,01,NaN,ALTO HOSPICIO-ALTO HOSPICIO-1-M,6,0.012048
2,IQUIQUE,1.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,3.0,PARTIDO REGIONALISTA DE LOS INDEPENDIENTES,RICARDO ISRAEL ZIPPER,2.0,01,NaN,ALTO HOSPICIO-ALTO HOSPICIO-1-M,6,0.012048
3,IQUIQUE,1.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,4.0,PARTIDO PROGRESISTA,MARCO ENRIQUEZ-OMINAMI GUMUCIO,15.0,01,NaN,ALTO HOSPICIO-ALTO HOSPICIO-1-M,4,0.090361
4,IQUIQUE,1.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,5.0,IGUALDAD,ROXANA MIRANDA MENESES,3.0,01,NaN,ALTO HOSPICIO-ALTO HOSPICIO-1-M,5,0.018072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372136,ARICA,1.0,1.0,ARICA,SAN MIGUEL DE AZAPA,23.0,NaN,5.0,IGUALDAD,ROXANA MIRANDA MENESES,1.0,15,NaN,ARICA-SAN MIGUEL DE AZAPA-23-NA,5,0.027027
372137,ARICA,1.0,1.0,ARICA,SAN MIGUEL DE AZAPA,23.0,NaN,6.0,PARTIDO SOCIALISTA DE CHILE,MICHELLE BACHELET JERIA,15.0,15,NaN,ARICA-SAN MIGUEL DE AZAPA-23-NA,1,0.405405
372138,ARICA,1.0,1.0,ARICA,SAN MIGUEL DE AZAPA,23.0,NaN,7.0,UNION DEMOCRATA INDEPENDIENTE,EVELYN MATTHEI FORNET,11.0,15,NaN,ARICA-SAN MIGUEL DE AZAPA-23-NA,2,0.297297
372139,ARICA,1.0,1.0,ARICA,SAN MIGUEL DE AZAPA,23.0,NaN,8.0,ECOLOGISTA VERDE,ALFREDO SFEIR YOUNIS,1.0,15,NaN,ARICA-SAN MIGUEL DE AZAPA-23-NA,5,0.027027


In [91]:
if year == 2021:

    df_location = df[["polling_id", "Nro. Región", "Región", "Provincia", "Circ. Senatorial", "Distrito", "Comuna", "Circ. Electoral", "Local",
        "Nro. Mesa", "Tipo de mesa", "Mesas Fusionadas", "Electores"]].drop_duplicates()

    df_location = df_location.rename(columns={
        "Nro. Región": "region_id",
        "Región": "region",
        "Provincia": "province",
        "Circ. Senatorial": "senate_district",
        "Distrito": "district",
        "Comuna": "commune",
        "Circ. Electoral": "electoral_circ",
        "Local": "polling_place",
        "Nro. Mesa": "polling_table",
        "Tipo de mesa": "polling_type",
        "Mesas Fusionadas": "mixed_polling",
        "Electores": "voters"
    })
    
    for col in ["region_id", "senate_district", "district", "polling_table", "voters"]:
        df_location[col] = df_location[col].astype(int)
    
elif year == 2017:
    df_location = df[["polling_id", "Región", "Provincia", "Circ.Senatorial", "Distrito", "Comuna", "Circ.Electoral", "Local",
        "Mesa", "Tipo mesa", "Mesas Fusionadas", "Electores"]].drop_duplicates()

    df_location = df_location.rename(columns={
        "Región": "region_id",
        "Provincia": "province",
        "Circ.Senatorial": "senate_district",
        "Distrito": "district",
        "Comuna": "commune",
        "Circ.Electoral": "electoral_circ",
        "Local": "polling_place",
        "Mesa": "polling_table",
        "Tipo mesa": "polling_type",
        "Mesas Fusionadas": "mixed_polling",
        "Electores": "voters"
    })
    
    for col in ["region_id", "polling_table", "voters"]:
        df_location[col] = df_location[col].astype(int)

elif year == 2013:
    df_location = df[["polling_id", "region_id", "Provincia", "Circ. Senatorial", "Distrito", "Comuna", "Circ. Electoral", 
        "Nro. Mesa", "Tipo Mesa"]].drop_duplicates()

    df_location = df_location.rename(columns={
        "Provincia": "province",
        "Circ. Senatorial": "senate_district",
        "Distrito": "district",
        "Comuna": "commune",
        "Circ. Electoral": "electoral_circ",
        "Nro. Mesa": "polling_table",
        "Tipo Mesa": "polling_type"
    })
    
    for col in ["region_id", "polling_table"]:
        df_location[col] = df_location[col].astype(int)

df_location.to_csv(f"data_output/Chile/{year}_{election_round}_location.csv.gz", compression="gzip", index=False)